<a href="https://colab.research.google.com/github/bimal-bp/Next-Word-Predictor-NLP-LTSM-Deep-Learning.ipynb/blob/ML_MODELS/Next_Word_Predictor_NLP_LTSM_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Specify the path to your file in Google Drive
file_path = '/content/drive/MyDrive/dataset (1).txt'

# Open and read the file
with open(file_path, 'r') as file:
    lines = file.readlines()

# Print specific lines from the file
print(lines[0])
print(lines[100])


﻿It was the best of times, it was the worst of times, it was the age of

coach across the road, with the mutinous intent of taking it back



In [6]:
# import libraies
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding,LSTM,Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

In [7]:
lines[:5]

['\ufeffIt was the best of times, it was the worst of times, it was the age of\n',
 'wisdom, it was the age of foolishness, it was the epoch of belief, it\n',
 'was the epoch of incredulity, it was the season of Light, it was the\n',
 'season of Darkness, it was the spring of hope, it was the winter of\n',
 'despair, we had everything before us, we had nothing before us, we were\n']

In [8]:
# cleaning
data=""
for i in lines:
  data=' '.join(lines)
data=data.replace('\n','').replace('\r','').replace('\ufeff','')
data[:200]

'It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of foolishness, it was the epoch of belief, it was the epoch of incredulity, it was the season of Light, i'

In [9]:
import string
translator=str.maketrans(string.punctuation, ' '*len(string.punctuation))
new_data=data.translate(translator)

new_data[:150]

'It was the best of times  it was the worst of times  it was the age of wisdom  it was the age of foolishness  it was the epoch of belief  it was the e'

In [10]:
z=[]
for i in data.split():
  if i not in z:
    z.append(i)
data=' '.join(z)
data[:150]

'It was the best of times, it worst age wisdom, foolishness, epoch belief, incredulity, season Light, Darkness, spring hope, winter despair, we had eve'

In [11]:
from IPython.core.displayhook import tokenize
# tokenization
tokenizer=Tokenizer()
tokenizer.fit_on_texts([data])

# saving the tiokenzier
pickle.dump(tokenizer,open('tokenizer.pkl','wb'))

sequence_data=tokenizer.texts_to_sequences([data])[0]
sequence_data[0:10]

[4, 19, 7, 815, 59, 332, 4, 816, 507, 2956]

In [12]:
vocab_size=len(tokenizer.word_index)+1
print(vocab_size)

8697


In [13]:
sequences=[]
for i in range(1,len(sequence_data)):
  words=sequence_data[i-1:i+1]
  sequences.append(words)
print("The length of sequences are : ",len(sequences))

sequences=np.array(sequences)
sequences[:10]

The length of sequences are :  17085


array([[   4,   19],
       [  19,    7],
       [   7,  815],
       [ 815,   59],
       [  59,  332],
       [ 332,    4],
       [   4,  816],
       [ 816,  507],
       [ 507, 2956],
       [2956, 2957]])

In [14]:
x=[]
y=[]
for i in sequences:
  x.append(i[0])
  y.append(i[1])

x=np.array(x)
y=np.array(y)

In [15]:
y=to_categorical(y,num_classes=vocab_size)
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [16]:
# model craete

model=Sequential()
model.add(Embedding(vocab_size,10,input_length=1))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(50,activation="relu"))
model.add(Dense(vocab_size,activation="softmax"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1, 10)             86970     
                                                                 
 lstm (LSTM)                 (None, 1, 50)             12200     
                                                                 
 lstm_1 (LSTM)               (None, 50)                20200     
                                                                 
 dense (Dense)               (None, 50)                2550      
                                                                 
 dense_1 (Dense)             (None, 8697)              443547    
                                                                 
Total params: 565467 (2.16 MB)
Trainable params: 565467 (2.16 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
# callbacks

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard

checkpoint=ModelCheckpoint("nextword1.h5",monitor='loss',verbose=1,
                           save_best_only=True,mode='auto')

reduce=ReduceLROnPlateau(monitor='loss',factor=0.2,patience=3,min_lr=0.0001,verbose=1)
logdir='logsnextword1'
tensorboard_visualization=TensorBoard(log_dir=logdir)

In [18]:
# compile
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001))

In [19]:
model.fit(x,y,epochs=5,batch_size=64,callbacks=[checkpoint, reduce, tensorboard_visualization])

Epoch 1/5
264/267 [============================>.] - ETA: 0s - loss: 8.9800
Epoch 1: loss improved from inf to 8.98075, saving model to nextword1.h5
267/267 [==============================] - 15s 14ms/step - loss: 8.9807 - lr: 0.0010
Epoch 2/5
 15/267 [>.............................] - ETA: 1s - loss: 8.6528

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


267/267 [==============================] - ETA: 0s - loss: 8.5064
Epoch 2: loss improved from 8.98075 to 8.50642, saving model to nextword1.h5
267/267 [==============================] - 2s 8ms/step - loss: 8.5064 - lr: 0.0010
Epoch 3/5
266/267 [============================>.] - ETA: 0s - loss: 8.2315
Epoch 3: loss improved from 8.50642 to 8.23167, saving model to nextword1.h5
267/267 [==============================] - 3s 11ms/step - loss: 8.2317 - lr: 0.0010
Epoch 4/5
262/267 [============================>.] - ETA: 0s - loss: 8.1008
Epoch 4: loss improved from 8.23167 to 8.10246, saving model to nextword1.h5
267/267 [==============================] - 3s 12ms/step - loss: 8.1025 - lr: 0.0010
Epoch 5/5
264/267 [============================>.] - ETA: 0s - loss: 8.0234
Epoch 5: loss improved from 8.10246 to 8.02669, saving model to nextword1.h5
267/267 [==============================] - 2s 8ms/step - loss: 8.0267 - lr: 0.0010


In [20]:
# prediction script
from tensorflow.keras.models import load_model
import numpy as np
import pickle

model=load_model('nextword1.h5')
tokenizer=pickle.load(open('tokenizer.pkl','rb'))

def predict_next_word(model,tokenizer,text):
  for i in range(3):
    sequence=tokenizer.texts_to_sequences([text][0])
    sequence=np.array(sequence)

    preds=model.predict_classes(sequences)

    for key,value in tokenizer.word_index.items():
      if value==preds:
        predicted_word=key

    print(predicted_word)
    return predicted_word

In [22]:
while(True):

  text=input("enter ur line : ")

  if text == "stop the script":
    print("end the progam")
    break
  else:
    try:
      text=text.split(" ")
      text=text[-1]

      text=' '.join(text)
      predict_next_word(model,tokenizer,text)
    except:

      continue

enter ur line : stop the script
end the progam
